In [ ]:
import sys
!{sys.executable} -m pip install spacy
!{sys.executable} -m pip install tensorflow
!{sys.executable} -m spacy download en_core_web_sm

In [80]:
from bom import featureEngineer
import keras
from keras import layers
from sklearn.model_selection import train_test_split

bomDf = featureEngineer('Example 3 BOM - Rack 1 and 4.csv')

columns_to_exclude = [
        'ISO', 'Spool', 'Subsystem', 'Pressure', 'Hydro Group', 'Module', 'EHT', 'Insul', 'Stage']
df = bomDf.drop(columns=columns_to_exclude, axis=1)

# Specify the features (X) and the target variable (y)
X = df.drop(columns=['Output Rack'])  
y = df['Output Rack']

# Split the data into training and testing sets
# Adjust the test_size parameter based on your preference
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Display the shapes of the resulting sets
print("X_train shape:", x_train.shape)
print("X_test shape:", x_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

# Determine input shape
input_shape = (X.shape[1],)  # Number of features in the DataFrame
input_var = keras.Input(shape=input_shape)

encoding_dim = X.shape[1]//4

# "encoded" is the encoded representation of the input
encoded = layers.Dense(encoding_dim, activation='relu')(input_var)
# "decoded" is the lossy reconstruction of the input
decoded = layers.Dense(X.shape[1], activation='tanh')(encoded)

X_train shape: (2354, 83)
X_test shape: (589, 83)
y_train shape: (2354,)
y_test shape: (589,)


In [81]:
# This model maps an input to its reconstruction
autoencoder = keras.Model(input_var, decoded)

In [82]:
# This model maps an input to its encoded representation
encoder = keras.Model(input_var, encoded)

In [83]:
# This is our encoded (32-dimensional) input
# encoding_dim = X.shape[1]//4
encoded_input = keras.Input(shape=(encoding_dim,))
# Retrieve the last layer of the autoencoder model
decoder_layer = autoencoder.layers[-1]
# Create the decoder model
decoder = keras.Model(encoded_input, decoder_layer(encoded_input))

In [84]:
# Compile the model using MSE loss function and Adam optimizer
autoencoder.compile(optimizer='adam', loss='categorical_crossentropy')

In [85]:
# Train for 50 epochs

# Have to change the type or else it won't be able to convert to tensor.
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

autoencoder.fit(x_train, x_train,
                epochs=50,
                batch_size=256,
                shuffle=True,
                validation_data=(x_test, x_test))

Epoch 1/50
10/10 [==============================] - 1s 20ms/step - loss: 10196.3301 - val_loss: 23204.9668
Epoch 2/50
10/10 [==============================] - 0s 5ms/step - loss: 23947.4707 - val_loss: 23181.4551
Epoch 3/50
10/10 [==============================] - 0s 6ms/step - loss: 23929.5078 - val_loss: 23170.1797
Epoch 4/50
10/10 [==============================] - 0s 6ms/step - loss: 7555.7441 - val_loss: 1164.3491
Epoch 5/50
10/10 [==============================] - 0s 5ms/step - loss: 1006.2512 - val_loss: 1175.2960
Epoch 6/50
10/10 [==============================] - 0s 5ms/step - loss: 964.3522 - val_loss: 826.9584
Epoch 7/50
10/10 [==============================] - 0s 5ms/step - loss: 838.0882 - val_loss: 864.5717
Epoch 8/50
10/10 [==============================] - 0s 5ms/step - loss: 8148.8506 - val_loss: 12786.5879
Epoch 9/50
10/10 [==============================] - 0s 5ms/step - loss: 18040.6074 - val_loss: 21377.1133
Epoch 10/50
10/10 [==============================] - 0s 8m

In [86]:
encoded = encoder.predict(x_test)
print(encoded)
print(encoded.shape)

19/19 [==============================] - 0s 1ms/step
[[  0.         0.         0.       ...   0.        35.31983  151.04398 ]
 [  0.         0.         0.       ...   0.        42.552746 185.90814 ]
 [  0.         0.         0.       ...   0.       100.82235  426.17404 ]
 ...
 [  0.         0.         0.       ...   0.       118.6686   497.85278 ]
 [  0.         0.         0.       ...   0.        56.064278 255.20044 ]
 [  0.         0.         0.       ...   0.        32.489513 152.2771  ]]
(589, 20)


In [ ]:
# decoded = decoder.predict(encoded)
# print(decoded)
# print(decoded.shape)

In [ ]:
# from sklearn.preprocessing import MinMaxScaler
# from sklearn.cluster import KMeans
# import pandas as pd

# num_clusters = 25

# kmeans = KMeans(n_clusters=num_clusters)

# sc = MinMaxScaler(feature_range=(0, 1))
# X = sc.fit_transform(encoded)

# kmeans.fit(X)
# # kmeans.fit(data[0])
# cluster_assignments = kmeans.labels_
# print(cluster_assignments)
# print(len(cluster_assignments))

# # Create a dictionary to store data points for each cluster
# clustered_data = {cluster_num: [] for cluster_num in range(num_clusters)}

# # Assign data points to their respective clusters
# for i in range(len(X)):
#     cluster_num = cluster_assignments[i]
#     row = bomDf[bomDf['ID'] == i]
#     clustered_data[cluster_num].append(row['Spool'].values)

# # print("Lengths of arrays:")

# # for key, value in clustered_data.items():
# #     print(f"{key}: {len(value)}")

# max_length = max(len(value) for value in clustered_data.values())

# # Fill in with None for arrays with shorter length
# clustered_data_filled = {
#     key: value + [None] * (max_length - len(value)) for key, value in clustered_data.items()}

# # print("Lengths of arrays:")

# # for key, value in clustered_data_filled.items():
# #     print(f"{key}: {len(value)}")

# df = pd.DataFrame(clustered_data_filled)

# df
# # Save the DataFrame to a CSV file
# # df.to_csv('Example3SpoolClusters.csv', index=False)